# ASR: Sphinx

In [5]:
import os
import wget
import tarfile
import argparse
import csv
from multiprocessing.pool import ThreadPool
import subprocess

## Download data

In [ ]:
target_dir = './'
os.makedirs(target_dir, exist_ok=True)

target_unpacked_dir = os.path.join(target_dir, "CV_unpacked")
os.makedirs(target_unpacked_dir, exist_ok=True)

if args.tar_path and os.path.exists(args.tar_path):
    print('Find existing file {}'.format(args.tar_path))
    target_file = args.tar_path
else:
    print("Could not find downloaded Common Voice archive, Downloading corpus...")
    filename = wget.download(COMMON_VOICE_URL, target_dir)
    target_file = os.path.join(target_dir, os.path.basename(filename))

print("Unpacking corpus to {} ...".format(target_unpacked_dir))
tar = tarfile.open(target_file)
tar.extractall(target_unpacked_dir)
tar.close()

## Data Preparation
1. Create ID files 
2. Create Transcription files
3. Convert to wav

In [ ]:
wav_dir = os.path.join(target_dir, 'wav/')
etc_dir = os.path.join(target_dir, 'etc/')
dataset_name = os.path.basename(target_dir)
os.makedirs(wav_dir, exist_ok=True)
os.makedirs(etc_dir, exist_ok=True)
path_to_data = os.path.dirname(csv_file)

In [ ]:
def create_fileids(csv_file, target_dir, dataset_type): 
    print('Creating fileids file for {}.'.format(csv_file))
    with open(csv_file) as csvfile:
        reader = csv.DictReader(csvfile)
        data = []
        with open(os.path.join(etc_dir, dataset_name + '_'+ dataset_type + '.fileids'), 'w') as ids_file:
            for row in reader: 
                file_name = row['filename']
                wav_path = os.path.join(wav_dir, dataset_type + '_'+ os.path.splitext(os.path.basename(file_name))[0] + '.wav')
                ids_file.write(os.path.basename(wav_path)[0] + '\n')

                data.append((file_name,wav_path))
        return data

In [ ]:
def create_transcript_files(csv_file, target_dir, dataset_type): 
    print('Creating transcript files for {}.'.format(csv_file))
    with open(csv_file) as csvfile:
        reader = csv.DictReader(csvfile)
        with open(os.path.join(etc_dir, dataset_name + '_'+ dataset_type + '.transcription'), 'w') as trans_file:
            for row in reader: 
                file_name = row['filename']
                wav_path = os.path.join(wav_dir, dataset_type + '_'+ os.path.splitext(os.path.basename(file_name))[0] + '.wav')
                trans_file.write('<s> '+ row['text'] + ' </s> (' + os.path.basename(wav_path)[0] + ')' + '\n')

In [ ]:
def convert_to_wav(x):
    file_path, wav_path = x
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    cmd = "sox {} -r {} -b 16 -c 1 {}".format(
        os.path.join(path_to_data, file_path),
        args.sample_rate,
        wav_path)
    subprocess.call([cmd], shell=True)

### Create training and validation data

In [ ]:
# Format the training data
train_csv_file = os.path.join(target_unpacked_dir, 'cv_corpus_v1/', 'cv-valid-train.csv'
train_wav_files = create_fileids(train_csv_file,target_dir, 'train')
create_transcript_files(train_csv_file,target_dir, 'train')
with ThreadPool(10) as pool:
    pool.map(convert_to_wav, train_wav_files)

# Format the validation data
val_csv_file = os.path.join(target_unpacked_dir, 'cv_corpus_v1/', 'cv-valid-dev.csv')
val_wav_files = create_fileids(val_csv_file, target_dir, 'test')
create_transcript_files(val_csv_file, target_dir, 'test')
with ThreadPool(10) as pool:
    pool.map(convert_to_wav, val_wav_files)

### Create word list from the training data

In [ ]:
import collections
import os

counter = collections.Counter()
with open(csv_file) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader: 
            trans = row['text']
            counter += collections.Counter(trans.split())

with open(os.path.join(etc_dir,'common_voice.words'), 'w') as f: 
    for item in counter:
        f.write(item.lower() + '\n')

### Create phonetic dictionary (lexicon) 
The phone list is the list of phones that will be used in the model. 
A default phone list is provided. 

A lexicon/phonetic dictionary can be obtained for each of the  words in the dataset by using: [CMU Lextool](http://www.speech.cs.cmu.edu/tools/lextool.html)

We provide one, to prevent some of the necessary formatting (convert to lower case, phone replacement to match phone list).

In [1]:
%%bash
cat etc/common_voice.phone

SIL
aa
ae
ah
ao
aw
ay
b
ch
d
dh
eh
er
ey
f
g
hh
ih
iy
jh
k
l
m
n
ng
ow
oy
p
r
s
sh
t
th
uh
uw
v
w
y
z
zh

In [2]:
%%bash
head etc/common_voice.dic

a	ah
a(2)	ey
monk	m ah ng k
dressed	d r eh s t
in	ih n
black	b l ae k
came	k ey m
to	t uw
to(2)	t ih
to(3)	t ah


### Create Language Model

In [ ]:
!ls etc/common_voice_train_transcript_only.txt

In [28]:
%%bash
# Create vocab file
text2wfreq < etc/common_voice_train_transcript_only.txt | wfreq2vocab > etc/common_voice.vocab

# # Create n-gram count from training transcript file
text2idngram -vocab etc/common_voice.vocab -idngram etc/common_voice.idngram < etc/common_voice_train.transcription 

# # Create language model from n-grams
idngram2lm -vocab_type 0 -idngram etc/common_voice.idngram -vocab etc/common_voice.vocab -arpa etc/common_voice.lm

# # Convert language model to DMP format
!sphinx_lm_convert -i etc/common_voice.lm -o etc/common_voice.lm.DMP

........


text2wfreq : Reading text from standard input...
wfreq2vocab : Will generate a vocabulary containing the most
              frequent 20000 words. Reading wfreq stream from stdin...
text2wfreq : Done.
wfreq2vocab : Done.
text2idngram
Vocab                  : etc/common_voice.vocab
Output idngram         : etc/common_voice.idngram
N-gram buffer size     : 100
Hash table size        : 2000000
Temp directory         : cmuclmtk-L4jI1w
Max open files         : 20
FOF size               : 10
n                      : 3
Initialising hash table...
Reading vocabulary... 
Allocating memory for the n-gram buffer...
Reading text into the n-gram buffer...
20,000 n-grams processed for each ".", 1,000,000 for each line.
..................................................
..................................................
.....................
Sorting n-grams...
Writing sorted n-grams to temporary file cmuclmtk-L4jI1w/1
Merging 1 temporary files...

2-grams occurring:	N times		> N times	Sug. -spec_num va

In [30]:
%%bash
sphinx_lm_convert -i etc/common_voice.lm -o etc/common_voice.dmp

Current configuration:
[NAME]		[DEFLT]	[VALUE]
-case			
-help		no	no
-i			etc/common_voice.lm
-ifmt			
-logbase	1.0001	1.000100e+00
-mmap		no	no
-o			etc/common_voice.dmp
-ofmt			

INFO: ngram_model_trie.c(354): Trying to read LM in trie binary format
INFO: ngram_model_trie.c(365): Header doesn't match
INFO: ngram_model_trie.c(177): Trying to read LM in arpa format
INFO: ngram_model_trie.c(193): LM of order 3
INFO: ngram_model_trie.c(195): #1-grams: 8005
INFO: ngram_model_trie.c(195): #2-grams: 31528
INFO: ngram_model_trie.c(195): #3-grams: 49969
INFO: lm_trie.c(474): Training quantizer
INFO: lm_trie.c(482): Building LM trie


In [3]:
%%bash
head etc/common_voice.lm

#############################################################################
## Copyright (c) 1996, Carnegie Mellon University, Cambridge University,
## Ronald Rosenfeld and Philip Clarkson
## Version 3, Copyright (c) 2006, Carnegie Mellon University 
## Contributors includes Wen Xu, Ananlada Chotimongkol, 
## David Huggins-Daines, Arthur Chan and Alan Black 
#############################################################################
===============  This file was produced by the CMU-Cambridge  ===============
===============     Statistical Language Modeling Toolkit     ===============


### Pre-processing complete
Make sure we have all the files necessary.
```
$ls etc/

common_voice.dic      common_voice.vocab
common_voice.filler   common_voice_test.fileids
common_voice.idngram  common_voice_test.transcription
common_voice.lm       common_voice_train.fileids
common_voice.lm.bin   common_voice_train.transcription
common_voice.phone
```

In [9]:
!ls etc/

common_voice.dic      common_voice.vocab
common_voice.filler   common_voice_test.fileids
common_voice.idngram  common_voice_test.transcription
common_voice.lm       common_voice_train.fileids
common_voice.lm.bin   common_voice_train.transcription
common_voice.phone


## Run Setup 
Creates the config for the training process

In [32]:
%%bash

sphinxtrain -t common_voice setup

Sphinxtrain path: /usr/local/lib/sphinxtrain
Sphinxtrain binaries path: /usr/local/libexec/sphinxtrain
Setting up the database common_voice


## Train Sphinx Model
Becasue the Sphinx is CPU-based, the training on a large dataset can take a very long time. 

In [33]:
import subprocess, time, os, sys
cmd = ["sphinxtrain","run"]

p = subprocess.Popen(cmd,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)

for line in iter(p.stdout.readline, b''):
    print(">>> " + line.decode().rstrip())

>>> MODULE: 000 Computing feature from audio files
>>> Extracting features from  segments starting at  (part 1 of 1)
>>> Extracting features from  segments starting at  (part 1 of 1)
>>> Feature extraction is done
>>> MODULE: 00 verify training files
>>>     Phase 1: Checking to see if the dict and filler dict agrees with the phonelist file.
>>>         Found 9156 words using 40 phones
>>>     Phase 2: Checking to make sure there are not duplicate entries in the dictionary
>>>     Phase 3: Check general format for the fileids file; utterance length (must be positive); files exist
>>>     Phase 4: Checking number of lines in the transcript file should match lines in fileids file
>>>     Phase 5: Determine amount of training data, see if n_tied_states seems reasonable.
>>>         Estimated Total Hours Training: 218.395580555556
>>>         100+ hours of training data is goodly amount of data.
>>>         Rule of thumb suggests 8000 for 100 hours, you can adjust accordingly.
>>>     Phas

>>>         w 0
>>>         w 1
>>>         w 2
>>>         y 0
>>>         y 1
>>>         y 2
>>>         z 0
>>>         z 1
>>>         z 2
>>>         zh 0
>>>         zh 1
>>>         zh 2
>>> MODULE: 45 Prune Trees
>>>     Phase 1: Tree Pruning
>>>     Phase 2: State Tying
>>> MODULE: 50 Training Context dependent models
>>>     Phase 1: Cleaning up directories:
>>> 	accumulator...logs...qmanager...
>>>     Phase 2: Copy CI to CD initialize
>>>     Phase 3: Forward-Backward
>>>         Baum welch starting for 1 Gaussian(s), iteration: 1 (1 of 1)
>>>         0% 10% 20% 30% 40% 50% 60% 70% 80% 90%
>>> ERROR: This step had 1646 ERROR messages and 0 WARNING messages.  Please check the log file for details.
>>>         Normalization for iteration: 1
>>>         Current Overall Likelihood Per Frame = -156.3750547187
>>>         Baum welch starting for 1 Gaussian(s), iteration: 2 (1 of 1)
>>>         0% 10% 20% 30% 40% 50% 60% 70% 80% 90%
>>> ERROR: This step had 1732 ERROR messages an

>>>         Normalization for iteration: 7
>>>         Current Overall Likelihood Per Frame = -154.299844716246
>>> Training for 8 Gaussian(s) completed after 7 iterations
>>> MODULE: 60 Lattice Generation
>>> Skipped:  $ST::CFG_MMIE set to 'no' in sphinx_train.cfg
>>> MODULE: 61 Lattice Pruning
>>> Skipped:  $ST::CFG_MMIE set to 'no' in sphinx_train.cfg
>>> MODULE: 62 Lattice Format Conversion
>>> Skipped:  $ST::CFG_MMIE set to 'no' in sphinx_train.cfg
>>> MODULE: 65 MMIE Training
>>> Skipped:  $ST::CFG_MMIE set to 'no' in sphinx_train.cfg
>>> MODULE: 90 deleted interpolation
>>> Skipped for continuous models
>>> MODULE: DECODE Decoding using models previously trained
>>>         Decoding 4076 segments starting at 0 (part 1 of 1)
>>>         0% ERROR: FATAL: "batch.c", line 913: PocketSphinx decoder init failed
>>> 
>>> ERROR: This step had 3 ERROR messages and 0 WARNING messages.  Please check the log file for details.
>>> ERROR: Failed to start pocketsphinx_batch
>>>         Alignin

In [37]:
!sphinx_lm_convert -i etc/common_voice.lm -o etc/common_voice.lm.DMP

Current configuration:
[NAME]		[DEFLT]	[VALUE]
-case			
-help		no	no
-i			etc/common_voice.lm
-ifmt			
-logbase	1.0001	1.000100e+00
-mmap		no	no
-o			etc/common_voice.lm.DMP
-ofmt			

INFO: ngram_model_trie.c(354): Trying to read LM in trie binary format
INFO: ngram_model_trie.c(365): Header doesn't match
INFO: ngram_model_trie.c(177): Trying to read LM in arpa format
INFO: ngram_model_trie.c(193): LM of order 3
INFO: ngram_model_trie.c(195): #1-grams: 8005
INFO: ngram_model_trie.c(195): #2-grams: 31528
INFO: ngram_model_trie.c(195): #3-grams: 49969
INFO: lm_trie.c(474): Training quantizer
INFO: lm_trie.c(482): Building LM trie


Need to change in config:
$CFG_CD_TRAIN = 'no';
$DEC_CFG_MODEL_NAME = "$CFG_EXPTNAME.ci_cont";


In [47]:
import subprocess, time, os, sys
cmd = ["sphinxtrain","-s", "decode", "run"]

p = subprocess.Popen(cmd,
                     stdout=subprocess.PIPE,
                     stderr=subprocess.STDOUT)

for line in iter(p.stdout.readline, b''):
    print(">>> " + line.decode().rstrip())

>>> MODULE: DECODE Decoding using models previously trained
>>>         Decoding 4076 segments starting at 0 (part 1 of 1)
>>>         0%
>>> ERROR: This step had 6 ERROR messages and 0 WARNING messages.  Please check the log file for details.
>>>         Aligning results to find error rate
>>> word_align.pl failed with error code 65280 at /usr/local/lib/sphinxtrain/scripts/decode/slave.pl line 173.
>>> Sphinxtrain path: /usr/local/lib/sphinxtrain
>>> Sphinxtrain binaries path: /usr/local/libexec/sphinxtrain


In [40]:
!ls 

Sphinx.ipynb	   etc	   model_architecture  result  wav
bwaccumdir	   feat    model_parameters    test
common_voice.html  logdir  qmanager	       trees


## Predict on test set

In [4]:
%%bash
pocketsphinx_batch  -adcin yes  -cepdir test/  -cepext .wav  -ctl test.fileids -hmm model_parameters/common_voice.ci_cont/ -lm etc/common_voice.lm.DMP -dict etc/common_voice.dic -hyp predictions.hyp

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
def wer(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
    s1 (string): space-separated sentence
    s2 (string): space-separated sentence
    """
    
    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))
    
    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]
    wer_lev = Lev.distance(''.join(w1), ''.join(w2))
    wer_inst = float(wer_lev)/len(s1.split()) * 100
    return 'WER: {0:.2f}'.format(wer_inst)

def cer(s1, s2):
    """
    Computes the Character Error Rate, defined as the edit distance.
    Arguments:
    s1 (string): space-separated sentence
    s2 (string): space-separated sentence
    """
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')
    cer_inst = float(Lev.distance(s1, s2)) / len(s1) * 100
    return 'CER: {0:.2f}'.format(cer_inst)

In [47]:
import Levenshtein as Lev
def word_lev(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]

    return Lev.distance(''.join(w1), ''.join(w2))

def char_lev(s1, s2):
    """
    Computes the Character Error Rate, defined as the edit distance.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """
    s1, s2, = s1.replace(' ', ''), s2.replace(' ', '')
    return Lev.distance(s1, s2)

In [91]:
import pandas as pd
target_data = pd.read_csv('cv-valid-test.csv')
predicted_data = pd.read_csv('predictions.hyp', names=['text'], header=None)
predicted_data['text'] = predicted_data['text'].str.replace(r"\(.*\)","")

In [71]:
index = 2
print(str(wer(predicted_data['text'][index], target_data['text'][index])))
print('Hypothesis: '+ predicted_data['text'][index] +'\nTarget: ' + target_data['text'][index])

WER: 100.00
Hypothesis: then the old to 
Target: and you know it


In [95]:
total_cer, total_wer, num_tokens, num_chars = 0, 0, 0, 0
verbose = True

for index in range(len(target_data)):
    transcript, reference = predicted_data['text'][index], target_data['text'][index]
    wer_inst = word_lev(transcript, reference)
    cer_inst = char_lev(transcript, reference)
    total_wer += wer_inst
    total_cer += cer_inst
    num_tokens += len(reference.split())
    num_chars += len(reference)
    if verbose:
        print("Ref:", reference.lower())
        print("Hyp:", transcript.lower())
        print("WER:", 100*float(wer_inst) / len(reference.split()), "CER:", 100*float(cer_inst) / len(reference), "\n")

Ref: without the dataset the article is useless
Hyp: the caravan said a word for six years 
WER: 114.28571428571429 CER: 66.66666666666667 

Ref: i've got to go to him
Hyp: i've got a good trip 
WER: 66.66666666666667 CER: 33.333333333333336 

Ref: and you know it
Hyp: then the old to 
WER: 100.0 CER: 73.33333333333333 

Ref: down below in the darkness were hundreds of people sleeping in peace
Hyp: down below in the darkness were hundreds of people sleeping in peace 
WER: 0.0 CER: 0.0 

Ref: hold your nose to keep the smell from disabling your motor functions
Hyp: hold your nose to keep the smell from disabling your motor functions 
WER: 0.0 CER: 0.0 

Ref: down below in the darkness were hundreds of people sleeping in peace
Hyp: down below in the darkness were hundreds of people sleeping in peace 
WER: 0.0 CER: 0.0 

Ref: strange images passed through my mind
Hyp: strange images passed through my mind 
WER: 0.0 CER: 0.0 

Ref: the sheep had taught him that
Hyp: the sheep had taught hi

Ref: even then he hardly understood what this meant
Hyp: he condemned the current saga boy this meant 
WER: 75.0 CER: 52.17391304347826 

Ref: i'd hate to have that many pins stuck in me
Hyp: i hate to have to make his destiny 
WER: 70.0 CER: 37.2093023255814 

Ref: i met a man with sharp nails
Hyp: mathematics rock playlist 
WER: 100.0 CER: 57.142857142857146 

Ref: only the lights in the great tent remained
Hyp: everything on this and i 
WER: 100.0 CER: 61.904761904761905 

Ref: apparently it's the dumplings that make this stew taste so good
Hyp: i need it to get to the sand 
WER: 100.0 CER: 63.492063492063494 

Ref: when she saw the unknown object bobbing up and down the edge she was absolutely horrified
Hyp: when she saw the unknown object bobbing up and arrange 
WER: 43.75 CER: 35.95505617977528 

Ref: i have known true alchemists the alchemist continued
Hyp: i'm going to happen got nothing to do 
WER: 100.0 CER: 67.3076923076923 

Ref: he really was a king a wise king
Hyp: the gr

WER: 16.666666666666668 CER: 8.695652173913043 

Ref: they placed the symbols of the pilgrimage on the doors of their houses
Hyp: but as the sun rose the corp paws that face 
WER: 84.61538461538461 CER: 60.0 

Ref: now i too will be one of the women who wait
Hyp: had our lives children into it 
WER: 100.0 CER: 58.13953488372093 

Ref: before the sun had reached its high point five hundred tribesmen appeared on the horizon
Hyp: because i cheat exactly gutted find coins and curiosity 
WER: 100.0 CER: 63.63636363636363 

Ref: others stumbled upon the stone by accident
Hyp: i've just told you to concentrate 
WER: 100.0 CER: 61.904761904761905 

Ref: what's all this mess
Hyp: all this mess 
WER: 25.0 CER: 30.0 

Ref: but now i'm sad and alone
Hyp: you have helped her mistakes minute ago 
WER: 116.66666666666667 CER: 112.0 

Ref: he asked it please never to stop speaking to him
Hyp: because of the suburban stop speaking to him 
WER: 60.0 CER: 35.416666666666664 

Ref: when you are loved you 

Hyp: i sold crystal and crossed the desert 
WER: 0.0 CER: 0.0 

Ref: looking down at the spoon he held the boy saw that the oil was gone
Hyp: but that's the jail boy saw a pair on is gone 
WER: 73.33333333333333 CER: 52.23880597014925 

Ref: so you got married
Hyp: so you got married 
WER: 0.0 CER: 0.0 

Ref: i still have no idea how to turn myself into the wind the boy repeated
Hyp: people are not able to transform himself into the boy repeated 
WER: 66.66666666666667 CER: 41.42857142857143 

Ref: in any case it's good that you've learned that everything in life has its price
Hyp: in the place is that the code you forget that everything in life has its price 
WER: 53.333333333333336 CER: 30.379746835443036 

Ref: maybe because that wasn't really his dream
Hyp: i have to be desired was really his dream 
WER: 85.71428571428571 CER: 40.476190476190474 

Ref: you should see the other guy specifically how good at fighting he is
Hyp: you should see the other guy specifically how good at fig

WER: 50.0 CER: 30.612244897959183 

Ref: i thought about the difficulties in translation that might arise
Hyp: paper going to go in translation that might arise 
WER: 50.0 CER: 37.5 

Ref: i found a little crowd of about twenty people surrounding the huge hole
Hyp: i found a little crowd of about twenty people surrounding the huge hole 
WER: 0.0 CER: 0.0 

Ref: so now i fear nothing because it was those omens that brought you to me
Hyp: somehow attempting because of the first clear i'm a sucker returning 
WER: 93.33333333333333 CER: 59.15492957746479 

Ref: i'm the leader of the caravan said a darkeyed bearded man
Hyp: finally a bath cannot said bother you can 
WER: 90.9090909090909 CER: 57.89473684210526 

Ref: and he decided he would go to see the chiefs of the tribes
Hyp: in spite of this thing 
WER: 92.3076923076923 CER: 60.3448275862069 

Ref: he had always believed that the sheep were able to understand what he said
Hyp: but he had always believed that the sheep were able to unde

...

Ref: and maybe it wasn't that they were teaching me but that i was learning from them
Hyp: immediately but it just teach me that i was learning from them 
WER: 50.0 CER: 37.5 

Ref: at that time it was quite clear in my own mind that the thing had come from the planet mars
Hyp: but that i wasn't prepared everyone mounted up out of the season 
WER: 90.0 CER: 57.142857142857146 

Ref: i went stretching my legs clumsily and feeling my way in the dark
Hyp: i went stretching my legs clumsily and feeling my way through 
WER: 23.076923076923077 CER: 10.76923076923077 

Ref: want to give my flower a drink
Hyp: but those who are trying 
WER: 100.0 CER: 56.666666666666664 

Ref: there was a shimmering green light emanating from the statue
Hyp: there was a shimmering green light emanating to do 
WER: 30.0 CER: 20.0 

Ref: were you paid for it or weren't you
Hyp: from the koran what are you 
WER: 87.5 CER: 57.142857142857146 

Ref: i don't know you from adam
Hyp: i don't know you from adam 
WER: 0

Ref: you must always know what it is that you want the old king had said
Hyp: he was going on why did you want one he had faith 
WER: 80.0 CER: 53.73134328358209 

Ref: that's what i'm going to do
Hyp: but i'm going to do 
WER: 33.333333333333336 CER: 33.333333333333336 

Ref: most meteorites are more or less rounded
Hyp: most meteorites are more or less rounded 
WER: 0.0 CER: 0.0 

Ref: my seven year resume gap is marked not drugs
Hyp: the kangaroo began to slow attracts 
WER: 100.0 CER: 61.36363636363637 

Ref: did you hear what i said
Hyp: the horizon 
WER: 100.0 CER: 62.5 

Ref: the criminals taunted the police officers and prison guards
Hyp: the criminals taunted the police officers and prison guards 
WER: 0.0 CER: 0.0 

Ref: the stranger was speaking of things that very few people knew about
Hyp: the stranger was speaking of things have anything to another 
WER: 50.0 CER: 31.34328358208955 

Ref: i myself heard nothing of that
Hyp: i myself heard nothing of that 
WER: 0.0 CER: 0.

Ref: i found a little crowd of about twenty people surrounding the huge hole
Hyp: the top and the garbage has to hunt the thought was invented 
WER: 100.0 CER: 64.78873239436619 

Ref: he thought of the merchant's daughter and was sure that she had probably married
Hyp: he got to produce my hamster in and see how they feel 
WER: 92.85714285714286 CER: 60.0 

Ref: i have the diet of a kid who found twenty dollars
Hyp: they began to travel but we can do 
WER: 100.0 CER: 57.142857142857146 

Ref: i have already found my treasure
Hyp: i have already found my treasure 
WER: 0.0 CER: 0.0 

Ref: the boy was shocked
Hyp: the boy was shocked 
WER: 0.0 CER: 0.0 

Ref: it spoke of journeys discoveries books and change
Hyp: it spoke of journeys discoveries want to change 
WER: 25.0 CER: 16.3265306122449 

Ref: the top of the cylinder was rotating on its body
Hyp: i can't think of any kind of bored 
WER: 100.0 CER: 62.5 

Ref: the navy's going to fly father home this afternoon
Hyp: the navy's going

WER: 53.333333333333336 CER: 38.57142857142857 

Ref: the meeting was over
Hyp: the meeting was over 
WER: 0.0 CER: 0.0 

Ref: they mounted their horses and rode out in the direction of the pyramids of egypt
Hyp: they mounted their horses aren't hundreds of egypt 
WER: 60.0 CER: 35.0 

Ref: it spoke of journeys discoveries books and change
Hyp: it spoke of journeys discoveries books and change 
WER: 0.0 CER: 0.0 

Ref: love is the force that transforms and improves the soul of the world
Hyp: was that for some patience and promises own story 
WER: 92.3076923076923 CER: 52.94117647058823 

Ref: lighthouses in a foggy world
Hyp: because of the road 
WER: 100.0 CER: 64.28571428571429 

Ref: you point to a trail of ants leading into the house
Hyp: you point to a trail makes heating him to the falls 
WER: 45.45454545454545 CER: 25.49019607843137 

Ref: the thing itself was almost entirely buried in sand
Hyp: the thing itself was almost unbearable merchant said 
WER: 44.44444444444444 CER: 29

WER: 75.0 CER: 40.90909090909091 

Ref: some went away while i was there and other people came
Hyp: it sounded like all said the old king 
WER: 100.0 CER: 61.111111111111114 

Ref: it has happened many times before
Hyp: that's comparing times before 
WER: 66.66666666666667 CER: 39.39393939393939 

Ref: you can't trust anybody in this crazy world
Hyp: you can't trust anybody else crystal 
WER: 50.0 CER: 27.906976744186046 

Ref: the shepherd swore that he would
Hyp: the shepherd swore that he would 
WER: 0.0 CER: 0.0 

Ref: i'll have it in a minute
Hyp: i'll have it in a minute 
WER: 0.0 CER: 0.0 

Ref: his hands began to tremble and the woman sensed it
Hyp: the man the ads catherine 
WER: 90.0 CER: 58.0 

Ref: they were looking only for gold his companion answered
Hyp: they were on the telephone gold his companion answered 
WER: 33.333333333333336 CER: 24.074074074074073 

Ref: she'll see us through dinner
Hyp: shame on state are so good to me 
WER: 160.0 CER: 71.42857142857143 

Ref: 

In [96]:
wer = float(total_wer) / num_tokens
cer = float(total_cer) / num_chars

print('Test Summary \t'
      'Average WER {wer:.3f}\t'
      'Average CER {cer:.3f}\t'.format(wer=wer * 100, cer=cer * 100))

Test Summary 	Average WER 53.849	Average CER 34.067	
